In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.display import display, HTML
from util import run_experiment, parse_stat, parse_stat_csv
import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
benchmarks = ['llama3-8b-d1']
traces = ['./hw_run/llm-inference/11.5']
gpus = {
    'A100_40GB': {
        'gpgpu_n_mem': 32,
        'gpgpu_n_clusters': 108,
        'gpgpu_clock_domains': '1410:1410:1410:607',
        'gpgpu_n_mem_per_ctrlr': 1,
        'gpgpu_dram_buswidth': 16,
        'gpgpu_dram_burst_length': 2,
    },
    'A100_80GB': {
        'gpgpu_n_mem': 40,
        'gpgpu_n_clusters': 108,
        'gpgpu_clock_domains': '1410:1410:1410:607',
        'gpgpu_n_mem_per_ctrlr': 1,
        'gpgpu_dram_buswidth': 16,
        'gpgpu_dram_burst_length': 2,
    },
    'H100_80GB': {
        'gpgpu_n_mem': 40,
        'gpgpu_n_clusters': 132,
        'gpgpu_clock_domains': '1830:1830:1830:800',
        'gpgpu_n_mem_per_ctrlr': 1,
        'gpgpu_dram_buswidth': 16,
        'gpgpu_dram_burst_length': 2,
    }
}

processes = []
for b, t, gpu in itertools.product(benchmarks, traces, gpus):
    name = f"{b.replace('-', '_')}_{gpu}-SASS"
    print(f'Running {name}')
    params = gpus[gpu]
    params['k'] = params['gpgpu_n_clusters'] + params['gpgpu_n_mem']
    proc = run_experiment(b, t, name, params, extra_args='-M 32G')
    processes.append(proc)

for p in processes:
    out, err = p.communicate()
    print(out)
    print(err)

Running llama3_8b_d1_A100_40GB-SASS
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B llama3-8b-d1 -C llama3_8b_d1_A100_40GB-SASS -T ./hw_run/llm-inference/11.5 -N llama3_8b_d1_A100_40GB-SASS -M 32G
Running llama3_8b_d1_A100_80GB-SASS


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B llama3-8b-d1 -C llama3_8b_d1_A100_80GB-SASS -T ./hw_run/llm-inference/11.5 -N llama3_8b_d1_A100_80GB-SASS -M 32G
Running llama3_8b_d1_H100_80GB-SASS
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B llama3-8b-d1 -C llama3_8b_d1_H100_80GB-SASS -T ./hw_run/llm-inference/11.5 -N llama3_8b_d1_H100_80GB-SASS -M 32G


your 131072x1 screen size is bogus. expect trouble


In [18]:
benchmarks = ['llama3-8b-d1']
traces = ['./hw_run/llm-inference/11.5']
n_mems = [32]

processes = []
for b, t, n_mem in itertools.product(benchmarks, traces, n_mems):
    name = f"{b.replace('-', '_')}_nmem={n_mem}-SASS"
    stat = parse_stat(b, name)
    print('\n'.join(f'{k}: {v}' for k, v in stat.items()))

your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


Accel-Sim-build: accelsim-commit-23daad2_modified_6.0_24-09-19- 1-43-42
GPGPU-Sim-build: gpgpu-sim_git-commit-67fc78cf_modified_6.0
gpu_tot_sim_insn\s*=\s*(.*): 2019142550
gpgpu_simulation_time\s*=.*\(([0-9]+) sec\).*: 7070
gpu_tot_sim_cycle\s*=\s*(.*): 2431956
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 1020157
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 8414720
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 135036
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 184840
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 93241392
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 57693576
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 61040
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 184840
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*): 27133096
gpgpu_n_tot_w_icount

In [19]:
metrics = [
    'gpu_tot_sim_cycle',
    'gpu_tot_ipc',
    'L2_BW_total',
    'gpu_tot_occupancy',
    'L1D_total_cache_miss_rate',
    'L2_total_cache_miss_rate',
    'dram_util',
    'num_tot_coalesced_accesses',
    'num_tot_uncoalesced_accesses',
    'num_coalesced_load',
    'num_coalesced_store',
    'num_uncoalesced_load',
    'num_uncoalesced_store',
]
df = pd.DataFrame()
df['metrics'] = metrics

for b, t, n_mem in itertools.product(benchmarks, traces, n_mems):
    name = f"{b.replace('-', '_')}_nmem={n_mem}-SASS"
    stat = parse_stat(b, name)
    val = []
    for m in metrics:
        for k, v in stat.items():
            if m in k:
                val.append(v)
                break
    df[f'{n_mem}'] = val


HTML(df.to_html(index=False))

your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


metrics,32
gpu_tot_sim_cycle,2431956
gpu_tot_ipc,830.2546
L2_BW_total,159.5474
gpu_tot_occupancy,33.7904
L1D_total_cache_miss_rate,0.0914
L2_total_cache_miss_rate,0.8566
dram_util,0.798734
num_tot_coalesced_accesses,93426232
num_tot_uncoalesced_accesses,343564544
num_coalesced_load,93241392


In [4]:
benchmarks = ['ece511-rodinia-3.1', 'ece511-parboil', 'ece511-polybench']
configs = ['ECE511_TITANV-SASS']*3
traces = ['./hw_run/rodinia-3.1/11.0', './hw_run/parboil/11.0', './hw_run/polybench/11.0']

processes = []
for b, c, t in zip(benchmarks, configs, traces):
    name = f'{b}-baseline'
    print(f'Running {name}')
    proc = run_experiment(b, c, t, name)
    processes.append(proc)

for p in processes:
    out, err = p.communicate()
    print(out)
    print(err)

Running ece511-rodinia-3.1-baseline
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-rodinia-3.1 -C ECE511_TITANV-SASS -T ./hw_run/rodinia-3.1/11.0 -N ece511-rodinia-3.1-baseline 
Running ece511-parboil-baseline


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-parboil -C ECE511_TITANV-SASS -T ./hw_run/parboil/11.0 -N ece511-parboil-baseline 
Running ece511-polybench-baseline
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-polybench -C ECE511_TITANV-SASS -T ./hw_run/polybench/11.0 -N ece511-polybench-baseline 


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


Cannot find a supported job management system. Spawning jobs locally.
Running Simulations with GPGPU-Sim built from 
accelsim-commit-2260456ea5e6a1420f5734f145a4b7d8ab1d4737_modified_4.5
 
Using configs: ECE511_TITANV-SASS
Benchmark: ece511-rodinia-3.1
Run Subdir = ECE511_TITANV-SASS
Parameters = 
#SASS
#SASS-Driven Accel-Sim

Base config file = /home/davidchencsl/code/accel-sim/gpu-simulator/gpgpu-sim/configs/tested-cfgs/SM7_TITANV/gpgpusim.config
Job 2 queued (hotspot-rodinia-3.1-1024_2_2___data_temp_1024___data_power_1024_output_out ECE511_TITANV-SASS)
ProcMan spawned [pid=29648]


Cannot find a supported job management system. Spawning jobs locally.
Running Simulations with GPGPU-Sim built from 
accelsim-commit-2260456ea5e6a1420f5734f145a4b7d8ab1d4737_modified_4.5
 
Using configs: ECE511_TITANV-SASS
Benchmark: ece511-parboil
Run Subdir = ECE511_TITANV-SASS
Parameters = 
#SASS
#SASS-Driven Accel-Sim

Base config file = /home/davidchencsl/code/accel-sim/gpu-simulator/gpgpu-sim/config

In [5]:
for b, c in zip(benchmarks, configs):
    stat = parse_stat(b, c)
    print('\n'.join(f'{k}: {v}' for k, v in stat.items()))

your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


Accel-Sim-build: accelsim-commit-2260456ea5e6a1420f5734f145a4b7d8ab1d4737_modified_4.5
GPGPU-Sim-build: gpgpu-sim_git-commit-67fc78cf_modified_6.0
gpu_tot_sim_insn\s*=\s*(.*): 372923416
gpgpu_simulation_time\s*=.*\(([0-9]+) sec\).*: 256
gpu_tot_sim_cycle\s*=\s*(.*): 68839
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 341523
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 695273
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 44027
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 175104
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 703696
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 1188
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 324
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 175104
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*): 7235
gpgpu_n_tot_w_icount\s*=

your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


Accel-Sim-build: accelsim-commit-2260456ea5e6a1420f5734f145a4b7d8ab1d4737_modified_4.5
GPGPU-Sim-build: gpgpu-sim_git-commit-67fc78cf_modified_6.0
gpu_tot_sim_insn\s*=\s*(.*): 1658916864
gpgpu_simulation_time\s*=.*\(([0-9]+) sec\).*: 943
gpu_tot_sim_cycle\s*=\s*(.*): 379164
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 1524924
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 2452480
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 135168
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 135168
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 9563136
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 546365
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 135168
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 135168
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*): 6564291
gpgpu_n_to

your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


Accel-Sim-build: accelsim-commit-2260456ea5e6a1420f5734f145a4b7d8ab1d4737_modified_4.5
GPGPU-Sim-build: gpgpu-sim_git-commit-67fc78cf_modified_6.0
gpu_tot_sim_insn\s*=\s*(.*): 620347492
gpgpu_simulation_time\s*=.*\(([0-9]+) sec\).*: 1766
gpu_tot_sim_cycle\s*=\s*(.*): 547834
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 1522053
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 3817918
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 0
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 2096128
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*): 21984780
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*): 7189016
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*): 0
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*): 2096128
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*): 10977846
gpgpu_n_tot_w_i

In [7]:
metrics = [
    'gpu_tot_sim_cycle',
    'gpu_tot_ipc',
    'L2_BW_total',
    'gpu_tot_occupancy',
    'L1D_total_cache_miss_rate',
    'L2_total_cache_miss_rate',
    'total dram reads',
    'total dram writes',
    'num_tot_coalesced_accesses',
    'num_tot_uncoalesced_accesses',
    'num_coalesced_load',
    'num_coalesced_store',
    'num_uncoalesced_load',
    'num_uncoalesced_store',
]
df = pd.DataFrame()
df['metrics'] = metrics

for b, c in zip(benchmarks, configs):
    stat = parse_stat(b, c)
    val = []
    for m in metrics:
        for k, v in stat.items():
            if m in k:
                val.append(v)
                break
    df[f'{b}'] = val


HTML(df.to_html(index=False))

your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


metrics,ece511-rodinia-3.1,ece511-parboil,ece511-polybench
gpu_tot_sim_cycle,68839,379164,547834
gpu_tot_ipc,5417.3276,4375.1963,1132.3640
L2_BW_total,485.5166,262.0652,414.5405
gpu_tot_occupancy,78.3181,41.1935,86.2327
L1D_total_cache_miss_rate,0.9900,0.2529,0.2456
L2_total_cache_miss_rate,0.4573,0.1524,0.7090
total dram reads,267019,394390,2097195
total dram writes,102526,63376,2056740
num_tot_coalesced_accesses,878800,9698304,24080908
num_tot_uncoalesced_accesses,4768841,77586432,167608360


In [5]:
import itertools
benchmarks = ['ece511-rodinia-3.1', 'ece511-parboil', 'ece511-polybench']
configs = ['ECE511_TITANV-SASS-SM_120', 'ECE511_TITANV-SASS-SM_160', 'ECE511_TITANV-SASS-SM_200']
traces = ['./hw_run/rodinia-3.1/11.0', './hw_run/parboil/11.0', './hw_run/polybench/11.0']

processes = []
for (b, t), c in itertools.product(zip(benchmarks,traces), configs):
    name = f'{b}-{c}'
    print(f'Running {name}')
    proc = run_experiment(b, c, t, name)
    processes.append(proc)

for p in processes:
    out, err = p.communicate()
    print(out)
    print(err)

Running ece511-rodinia-3.1-ECE511_TITANV-SASS-SM_120
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-rodinia-3.1 -C ECE511_TITANV-SASS-SM_120 -T ./hw_run/rodinia-3.1/11.0 -N ece511-rodinia-3.1-ECE511_TITANV-SASS-SM_120
Running ece511-rodinia-3.1-ECE511_TITANV-SASS-SM_160


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-rodinia-3.1 -C ECE511_TITANV-SASS-SM_160 -T ./hw_run/rodinia-3.1/11.0 -N ece511-rodinia-3.1-ECE511_TITANV-SASS-SM_160
Running ece511-rodinia-3.1-ECE511_TITANV-SASS-SM_200
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-rodinia-3.1 -C ECE511_TITANV-SASS-SM_200 -T ./hw_run/rodinia-3.1/11.0 -N ece511-rodinia-3.1-ECE511_TITANV-SASS-SM_200
Running ece511-parboil-ECE511_TITANV-SASS-SM_120


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-parboil -C ECE511_TITANV-SASS-SM_120 -T ./hw_run/parboil/11.0 -N ece511-parboil-ECE511_TITANV-SASS-SM_120
Running ece511-parboil-ECE511_TITANV-SASS-SM_160
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-parboil -C ECE511_TITANV-SASS-SM_160 -T ./hw_run/parboil/11.0 -N ece511-parboil-ECE511_TITANV-SASS-SM_160
Running ece511-parboil-ECE511_TITANV-SASS-SM_200


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-parboil -C ECE511_TITANV-SASS-SM_200 -T ./hw_run/parboil/11.0 -N ece511-parboil-ECE511_TITANV-SASS-SM_200
Running ece511-polybench-ECE511_TITANV-SASS-SM_120
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-polybench -C ECE511_TITANV-SASS-SM_120 -T ./hw_run/polybench/11.0 -N ece511-polybench-ECE511_TITANV-SASS-SM_120
Running ece511-polybench-ECE511_TITANV-SASS-SM_160


your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble
your 131072x1 screen size is bogus. expect trouble


/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-polybench -C ECE511_TITANV-SASS-SM_160 -T ./hw_run/polybench/11.0 -N ece511-polybench-ECE511_TITANV-SASS-SM_160
Running ece511-polybench-ECE511_TITANV-SASS-SM_200
/home/davidchencsl/miniconda3/envs/gem5/bin/python ./util/job_launching/run_simulations.py -B ece511-polybench -C ECE511_TITANV-SASS-SM_200 -T ./hw_run/polybench/11.0 -N ece511-polybench-ECE511_TITANV-SASS-SM_200
Cannot find a supported job management system. Spawning jobs locally.
Running Simulations with GPGPU-Sim built from 
accelsim-commit-2260456ea5e6a1420f5734f145a4b7d8ab1d4737_modified_0.0
 
Using configs: ECE511_TITANV-SASS-SM_120
Benchmark: ece511-rodinia-3.1
Run Subdir = ECE511_TITANV-SASS-SM_120
Parameters = 
#SASS
#SASS-Driven Accel-Sim

#SM_120
-gpgpu_n_clusters 60 -gpgpu_n_cores_per_cluster 2

Base config file = /home/davidchencsl/code/accel-sim/gpu-simulator/gpgpu-sim/configs/tested-cfgs/SM7_TITANV/gpgpusim.conf

In [15]:
df = pd.DataFrame()
df['metrics'] = metrics

for b, c in itertools.product(benchmarks, configs):
    stat = parse_stat(b, c)
    val = []
    for m in metrics:
        for k, v in stat.items():
            if m in k:
                val.append(v)
                break
    df[f'{b} {c}'] = val


HTML(df.to_html(index=False))

NameError: name 'metrics' is not defined